In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
submission_data = pd.read_csv('data/submission.csv')

In [3]:
submission_data.drop_duplicates(inplace=True)

In [4]:
submission_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7116 entries, 0 to 7115
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        7116 non-null   int64  
 1   접수연도              7116 non-null   int64  
 2   자치구코드             7116 non-null   int64  
 3   자치구명              7116 non-null   object 
 4   법정동코드             7116 non-null   int64  
 5   법정동명              7116 non-null   object 
 6   지번구분              6879 non-null   float64
 7   지번구분명             6879 non-null   object 
 8   본번                6879 non-null   float64
 9   부번                6879 non-null   float64
 10  건물명               6879 non-null   object 
 11  계약일               7116 non-null   int64  
 12  건물면적(㎡)           7116 non-null   float64
 13  토지면적(㎡)           6612 non-null   float64
 14  층                 6879 non-null   float64
 15  권리구분              65 non-null     object 
 16  취소일               261 non-null    float64


In [5]:
submission_data.drop(['Unnamed: 0','접수연도', '자치구명', '법정동명', '지번구분명', '본번', '부번', '취소일', '신고한 개업공인중개사 시군구명'], axis = 1, inplace = True)

In [6]:
submission_data['지번구분'].fillna(1, inplace=True)

In [7]:
def fill_missing_build(row):
    if pd.isnull(row['건축년도']):
        matching_values = submission_data[(submission_data['자치구코드'] == row['자치구코드']) & 
        (submission_data['지번구분'] == row['지번구분']) & (submission_data['법정동코드'] == row['법정동코드'])]['지번구분'].mode()
        if not matching_values.empty:
            return matching_values[0]
    return row['건축년도']

submission_data['건축년도'] = submission_data.apply(fill_missing_build, axis=1)

In [8]:
def fill_missing_build(row):
    if (row['건축년도']==0):
        matching_values = submission_data[(submission_data['자치구코드'] == row['자치구코드']) & 
        (submission_data['지번구분'] == row['지번구분']) & (submission_data['법정동코드'] == row['법정동코드'])]['건축년도'].mode()
        if not matching_values.empty:
            return matching_values[0]
    return row['건축년도']

submission_data['건축년도'] = submission_data.apply(fill_missing_build, axis=1)

In [9]:
submission_data['층'] = submission_data['층'].fillna(2)

In [10]:
conditions = [
    submission_data['층'] < 0,                       # '지하' 조건
    (submission_data['층'] >= 0) & (submission_data['층'] <= 10), # '저층' 조건
    (submission_data['층'] > 10) & (submission_data['층'] <= 20), # '중층' 조건
    (submission_data['층'] > 20) & (submission_data['층'] <= 30), # '고층' 조건
    submission_data['층'] > 30                       # '초고층' 조건
]
choices = [1, 2, 3, 4, 5]
submission_data['층_범주'] = np.select(conditions, choices, default=np.nan)
submission_data.drop(['층'], axis = 1, inplace = True)

In [11]:
mean_values = submission_data.groupby('자치구코드')['물건금액(만원)'].mean()
mean_values_sorted = mean_values.sort_values(ascending=True).reset_index()
mean_values_sorted['자치구코드_범주화'] = (mean_values_sorted.index // 5) + 1
submission_data = submission_data.merge(mean_values_sorted[['자치구코드', '자치구코드_범주화']], on='자치구코드', how='left')

In [12]:
mean_values2 = submission_data.groupby('법정동코드')['물건금액(만원)'].mean()
mean_values_sorted2 = mean_values2.sort_values(ascending=True).reset_index()
mean_values_sorted2['법정동코드_범주화'] = (mean_values_sorted2.index // 5) + 1
submission_data = submission_data.merge(mean_values_sorted2[['법정동코드', '법정동코드_범주화']], on='법정동코드', how='left')

In [13]:
cols = ["물건금액(만원)", "건물면적(㎡)", "토지면적(㎡)", "건축년도"]
for col in cols:
  submission_data = submission_data[np.abs(submission_data[col] - submission_data[col].mean()) <= (3*submission_data[col].std())]

In [14]:
submission_data['계약년월'] = submission_data['계약일'].astype(str).str[:6].astype('int64')
submission_data.drop(['계약일'], axis=1, inplace=True)

In [15]:
submission_data['물건금액_log'] = np.log1p(submission_data['물건금액(만원)'])
submission_data['건물면적_log'] = np.log1p(submission_data['건물면적(㎡)'])
submission_data['토지면적_log'] = np.log1p(submission_data['토지면적(㎡)'])

In [16]:
submission_data.drop(columns = ['건물면적(㎡)','토지면적(㎡)'], inplace=True)

In [17]:
submission_data.drop(['건물명'], axis=1, inplace=True)

In [18]:
submission_data.drop(['지번구분'], axis=1, inplace=True)

In [19]:
submission_data.isnull().sum()

자치구코드           0
법정동코드           0
권리구분         6350
건축년도            0
건물용도            0
신고구분            0
물건금액(만원)        0
층_범주            0
자치구코드_범주화       0
법정동코드_범주화       0
계약년월            0
물건금액_log        0
건물면적_log        0
토지면적_log        0
dtype: int64

In [20]:
submission_data = pd.get_dummies(submission_data, columns=['권리구분'], prefix='권리구분')
submission_data = pd.get_dummies(submission_data, columns=['신고구분'], prefix='신고구분')
submission_data = pd.get_dummies(submission_data, columns=['건물용도'], prefix='건물용도')

In [21]:
submission_data.isnull().sum()

자치구코드         0
법정동코드         0
건축년도          0
물건금액(만원)      0
층_범주          0
자치구코드_범주화     0
법정동코드_범주화     0
계약년월          0
물건금액_log      0
건물면적_log      0
토지면적_log      0
권리구분_분양권      0
권리구분_입주권      0
신고구분_중개거래     0
신고구분_직거래      0
건물용도_단독다가구    0
건물용도_아파트      0
건물용도_연립다세대    0
건물용도_오피스텔     0
dtype: int64

In [22]:
submission_data.to_csv('submission_preprocessed.csv', index=False)

In [23]:
submission_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6389 entries, 0 to 7115
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   자치구코드       6389 non-null   int64  
 1   법정동코드       6389 non-null   int64  
 2   건축년도        6389 non-null   float64
 3   물건금액(만원)    6389 non-null   int64  
 4   층_범주        6389 non-null   float64
 5   자치구코드_범주화   6389 non-null   int64  
 6   법정동코드_범주화   6389 non-null   int64  
 7   계약년월        6389 non-null   int64  
 8   물건금액_log    6389 non-null   float64
 9   건물면적_log    6389 non-null   float64
 10  토지면적_log    6389 non-null   float64
 11  권리구분_분양권    6389 non-null   bool   
 12  권리구분_입주권    6389 non-null   bool   
 13  신고구분_중개거래   6389 non-null   bool   
 14  신고구분_직거래    6389 non-null   bool   
 15  건물용도_단독다가구  6389 non-null   bool   
 16  건물용도_아파트    6389 non-null   bool   
 17  건물용도_연립다세대  6389 non-null   bool   
 18  건물용도_오피스텔   6389 non-null   bool   
dtypes: bool(8), float64(5), int64(6)